In [ ]:
!pip install -U lightning -q

# Neural Network Beginnings

In the world of machine learning and artificial intelligence, neural networks are a class of models inspired by the human brain. They are composed of layers of nodes, or "neurons", with each layer transforming its input data in a way that allows the network to learn from experience. Initially, neural networks were introduced to allow computers to recognize patterns, but they have since become a fundamental tool for a wide range of machine learning tasks.

![Neuron Diagram](single-neuron.jpg)

## A Single Neuron

A single neuron, also known as a perceptron, is the basic unit of a neural network. It takes a set of numerical inputs, combines them in a weighted sum, and passes them through an activation function to produce an output. Here's a breakdown of these components:

- **Inputs:** These are the data that are fed into the neuron. For example, in a neural network that processes images, the inputs could be the pixel values of an image.

- **Weights:** Each input is associated with a weight that represents its importance. The neuron learns the correct weights during the training process. Initially, these weights are often set randomly.

- **Biases:** The bias allows the neuron to have some flexibility in activation. It is a constant term that is added to the weighted sum of the inputs before the activation function is applied. Like the weights, biases are learned during training.

- **Activation Function:** After the weighted sum of the inputs and the bias are calculated, this sum is passed through an activation function. The purpose of this function is to introduce non-linearity into the system, allowing the neural network to learn from the error of its predictions and make necessary corrections. Common activation functions include ReLU (Rectified Linear Unit), Sigmoid, and Tanh.

In mathematical terms, the output $y$ of a single neuron can be described as follows:
$y = f(w_1 x_1 + w_2 x_2 + \cdots + w_n x_n + b)$
where:
$w_1, w_2, \ldots, w_n$ are the weights,
$x_1, x_2, \ldots, x_n$ are the inputs,
$b$ is the bias, and
$f$ is the activation function.

This simple structure, when scaled to hundreds or thousands of neurons arranged in layers, enables neural networks to approximate complex and highly non-linear functions, making them a powerful tool in machine learning.


In [ ]:
import numpy
import pytorch_lightning
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class Perceptron(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        # Define the single linear layer (4 input features, 1 output)
        self.layer = nn.Linear(4, 1)

    def forward(self, x):
        # Pass the input through the linear layer
        # then through the sigmoid activation function
        return torch.sigmoid(self.layer(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y.view(-1, 1).float())
        self.log("train_loss", loss)  # Logging the loss
        return loss

    def configure_optimizers(self):
        # We will use the Adam optimizer
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
# Generate synthetic data for the example
numpy.random.seed(42)
N = 1000
temp = numpy.random.uniform(0, 100, N)
rain = numpy.random.randint(0, 2, N)
ride = numpy.random.randint(0, 2, N)
friends = numpy.random.randint(0, 2, N)
X = numpy.vstack([temp, rain, ride, friends]).T
# Assume that the individual will go to the zoo if there is a ride and friends are going
Y = (ride & friends).astype(numpy.float32)

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Create TensorDatasets and DataLoaders
train_data = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(Y_train, dtype=torch.float32),
)
val_data = TensorDataset(
    torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32)
)
train_loader = DataLoader(train_data, batch_size=32)
val_loader = DataLoader(val_data, batch_size=32)

In [ ]:
# Start TensorBoard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
# Train the model
model = Perceptron()
trainer = pytorch_lightning.Trainer(max_epochs=50, log_every_n_steps=3)
trainer.fit(model, train_loader)

In [ ]:
# Generate some new data to test
test_temp = numpy.array([75, 85, 60, 90, 72])
test_rain = numpy.array([0, 1, 0, 1, 0])
test_ride = numpy.array([1, 0, 1, 0, 1])
test_friends = numpy.array([1, 1, 0, 0, 1])
X_test = numpy.vstack([test_temp, test_rain, test_ride, test_friends]).T

# Convert to a PyTorch tensor
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    predictions = model(X_test_tensor)

# Print predictions
for i, pred in enumerate(predictions):
    print(
        f"Conditions: {X_test[i]} | Probability of going to the zoo: {pred.item():.4f}"
    )